# Affiliations report summary

In [1]:
from datetime import datetime
start = datetime.utcnow() # For measuring the total processing time

In [2]:
import json
from urllib.request import urlopen
import pandas as pd
import numpy as np

/home/ednilson/.virtualenvs/jupyter/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Get collection information from ArticleMeta 

In [3]:
AMC_URL = "http://articlemeta.scielo.org/api/v1/collection/identifiers/"
amc_data = pd.DataFrame(json.load(urlopen(AMC_URL)))

In [4]:
amc_data.head(6)

,acron,acron2,code,document_count,domain,has_analytics,is_active,journal_count,name,original_name,status,type
0,arg,ar,arg,39007.0,www.scielo.org.ar,True,True,"{'current': 125, 'deceased': 22}","{'pt': 'Argentina', 'en': 'Argentina', 'es': '...",Argentina,certified,journals
1,chl,cl,chl,63467.0,www.scielo.cl,True,True,"{'current': 105, 'deceased': 13, 'suspended': 1}","{'pt': 'Chile', 'en': 'Chile', 'es': 'Chile'}",Chile,certified,journals
2,col,co,col,69725.0,www.scielo.org.co,True,True,"{'current': 226, 'suspended': 7}","{'pt': 'Colombia', 'en': 'Colombia', 'es': 'Co...",Colombia,certified,journals
3,cub,cu,cub,33492.0,scielo.sld.cu,True,True,"{'current': 61, 'deceased': 2, 'suspended': 4}","{'pt': 'Cuba', 'en': 'Cuba', 'es': 'Cuba'}",Cuba,certified,journals
4,esp,es,esp,37862.0,scielo.isciii.es,True,True,"{'current': 43, 'deceased': 6, 'suspended': 11}","{'pt': 'Espanha', 'en': 'Spain', 'es': 'España'}",España,certified,journals
5,mex,mx,mex,64406.0,www.scielo.org.mx,True,True,"{'current': 159, 'deceased': 12, 'suspended': 44}","{'pt': 'Mexico', 'en': 'Mexico', 'es': 'Mexico'}",Mexico,certified,journals


Some collections won't be analyzed, mainly to avoid duplicates
(there are articles in more than one collection).
The `spa` (*Public Health* collection) should have part of it
kept in the result, but it's not a collection
whose journals/articles are assigned to a single country.
The collections below are linked to a single country:

In [5]:
dont_evaluate = ["bio", "cci", "cic", "ecu", "psi", "pry", "rve", "rvo", "rvt", "sss", "spa", "wid"]
amc_names_map = {
    "code": "collection",
    "acron2": "origin",
}
amc_pairs = amc_data \
    [(amc_data["acron2"].str.len() == 2) &
     ~amc_data["code"].isin(dont_evaluate)] \
    [list(amc_names_map.keys())] \
    .rename(columns=amc_names_map) \
    .assign(origin=lambda df: df["origin"].str.upper())
amc_pairs

,collection,origin
0,arg,AR
1,chl,CL
2,col,CO
3,cub,CU
4,esp,ES
5,mex,MX
6,prt,PT
8,scl,BR
11,sza,ZA
12,ven,VE


## ISSN selection from `spa`

These journals in the `spa` collection have the following countries:

In [6]:
spa_issn_country = pd.DataFrame([
    ("0021-2571", "IT"),
    ("0042-9686", "CH"),
    ("1020-4989", "US"),
    ("1555-7960", "US"),
], columns=["issn", "origin"])
spa_issn_country # For collection = "spa", only!

,issn,origin
0,0021-2571,IT
1,0042-9686,CH
2,1020-4989,US
3,1555-7960,US


## Affiliations dataset

This dataset is the
[Network spreadsheet/CSV pack](https://static.scielo.org/tabs/tabs_network.zip)
 which can be found in the
[SciELO Analytics report](https://analytics.scielo.org/w/reports)
web page.
The first two rows of it are:

#### Unzip the CSV file

In [7]:
import zipfile
# Use the Zip file in jcatalog/data/scielo
# with zipfile.ZipFile('../../data/scielo/tabs_network_181203.zip', 'r') as zip_ref:
#     zip_ref.extract('documents_affiliations.csv', 'csv_files')
with zipfile.ZipFile('../../data/scielo/tabs_bra_190123.zip', 'r') as zip_ref:
    zip_ref.extract('documents_affiliations.csv', 'csv_files')

In [8]:
dataset = pd.read_csv("csv_files/documents_affiliations.csv", keep_default_na=False)
dataset.head(3).T

,0,1,2
extraction date,2019-01-22,2019-01-22,2019-01-22
study unit,document,document,document
collection,scl,scl,scl
ISSN SciELO,0100-879X,0100-879X,0100-879X
ISSN's,0100-879X;1414-431X,0100-879X;1414-431X,0100-879X;1414-431X
title at SciELO,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
title thematic areas,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences
title is agricultural sciences,0,0,0
title is applied social sciences,0,0,0
title is biological sciences,1,1,1


We won't need all the information,
and we can simplify the column names
for the columns we need:

In [9]:
names_map = {
    "document publishing ID (PID SciELO)": "pid",
    "document affiliation country ISO 3166": "country",
    "document is citable": "is_citable",
    "ISSN SciELO": "issn",
    "collection": "collection",
    "document publishing year": "year",
}
cdf = dataset[list(names_map.keys())].rename(columns=names_map)
cdf[610_000::80_000] # cdf stands for "Country/Collection Data Frame"

,pid,country,is_citable,issn,collection,year
610000,S0104-07072015000200399,BR,1,0104-0707,scl,2015
690000,S0080-62342016000500792,BR,1,0080-6234,scl,2016
770000,S0101-33002017000300051,BR,1,0101-3300,scl,2017


## Adding journal country as `origin`

The `country` column in the last dataframe is the affiliation country,
not the journal/article origin country.
Let's add the former as a new `origin` column,
grabbing it from the collection
or from the ISSN (when collection is `spa`):

In [10]:
cdfwof = pd.concat([
    pd.merge(cdf[cdf["collection"] != "spa"], amc_pairs,        how="inner", on="collection"),
    pd.merge(cdf[cdf["collection"] == "spa"], spa_issn_country, how="inner", on="issn"),
])
cdfwof[610_000::80_000] # wof stands for "With Origin, Filtered"

/home/ednilson/.virtualenvs/jupyter/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,collection,country,is_citable,issn,origin,pid,year
610000,scl,BR,1,0104-0707,BR,S0104-07072015000200399,2015
690000,scl,BR,1,0080-6234,BR,S0080-62342016000500792,2016
770000,scl,BR,1,0101-3300,BR,S0101-33002017000300051,2017


The rows without an assignable origin have been removed:

### Add years

In [11]:
cdfwof["years"] = np.where(cdfwof['year'] <= 2007, 'ate_2007', cdfwof["year"])

In [12]:
cdf.shape

(827839, 6)

In [13]:
cdfwof.shape

(827839, 8)

In [14]:
cdfwof[(cdfwof["pid"] == "S0004-27302009000900010")]

,collection,country,is_citable,issn,origin,pid,year,years
286000,scl,BR,1,0004-2730,BR,S0004-27302009000900010,2009,2009
286001,scl,BR,1,0004-2730,BR,S0004-27302009000900010,2009,2009
286002,scl,BR,1,0004-2730,BR,S0004-27302009000900010,2009,2009


## Country summary

Are the affiliations countries and the journal/origin country always the same?
The goal now is to create a summary of the affiliation countries
by comparing them to the journal/origin country.

In [15]:
origin_country = cdfwof["country"] == cdfwof["origin"]

In [16]:
result = cdfwof.assign(
    origin_country=origin_country,
    other_country=~(origin_country | (cdfwof["country"] == "")),
    no_country=cdfwof["country"] == "",
).groupby("pid").sum().assign(
    has_origin=lambda df: df["origin_country"].apply(bool),
    has_other=lambda df: df["other_country"].apply(bool),
    has_no=lambda df: df["no_country"].apply(bool),
).assign(
    has_both=lambda df: df["has_origin"] & df["has_other"],
    all_no=lambda df: ~(df["has_origin"] | df["has_other"]),
).applymap(int)

In [17]:
result[:20_000:2_500]

,is_citable,year,origin_country,other_country,no_country,has_origin,has_other,has_no,has_both,all_no
pid,,,,,,,,,,
S0001-37141998000300001,1,1998,0,0,1,0,0,1,0,1
S0002-05912013000100008,2,4026,0,0,2,0,0,1,0,1
S0004-27491995000600452,3,5985,3,0,0,1,0,0,0,0
S0004-27492014000600019,3,6042,3,0,0,1,0,0,0,0
S0004-282X1963000300003,1,1963,1,0,0,1,0,0,0,0
S0004-282X1999000100026,1,1999,0,0,1,0,0,1,0,1
S0004-282X2009000400041,1,2009,0,0,1,0,0,1,0,1
S0006-87051956000100016,1,1956,0,0,1,0,0,1,0,1


Each row has an affiliation summary for a single article,
identified by its PID.
A brief explanation of the columns:

* `origin_country`: Number of affiliations whose country is the origin country;
* `other_country`: Number of affiliations whose country isn't the origin country;
* `no_country`: Number of affiliations whose country is unknown;
* `has_origin`: This article has at least one affiliation whose country is the origin country;
* `has_other`: This article has at least one affiliation whose country isn't the origin country;
* `has_no`: This article has at least one affiliation whose country is unknown;
* `has_both`: This article has affiliations from both the origin country and another country;
* `all_no`: All affiliations are from unknown countries.

The trailing columns are represented by the integers
`1` (meaning `True`) and `0` (meaning `False`).

## Final result

Let's join the ISSN, collection and origin information to our analysis:

In [18]:
full_result = \
    pd.merge(result.reset_index(),
             cdfwof[["pid", "issn", "collection", "origin", "is_citable", "years"]].drop_duplicates(),
             how="left", on="pid") \
      .set_index("pid") \
      .sort_index()
full_result[7_500::30_000]

,is_citable_x,year,origin_country,other_country,no_country,has_origin,has_other,has_no,has_both,all_no,issn,collection,origin,is_citable_y,years
pid,,,,,,,,,,,,,,,
S0004-27492014000600019,3,6042,3,0,0,1,0,0,0,0,0004-2749,scl,BR,1,2014
S0034-75902017000100037,3,6051,1,2,0,1,1,0,1,0,0034-7590,scl,BR,1,2017
S0074-02762003000900014,1,2003,1,0,0,1,0,0,0,0,0074-0276,scl,BR,1,ate_2007
S0100-55022016000100077,3,6048,3,0,0,1,0,0,0,0,0100-5502,scl,BR,1,2016
S0101-74382003000100013,3,6009,1,0,2,1,0,1,0,0,0101-7438,scl,BR,1,ate_2007
S0102-695X2015000600569,2,4030,2,0,0,1,0,0,0,0,0102-695X,scl,BR,1,2015
S0103-65132017000200317,3,6051,1,2,0,1,1,0,1,0,0103-6513,scl,BR,1,2017
S0104-44782008000100007,1,2008,0,0,1,0,0,1,0,1,0104-4478,scl,BR,1,2008
S1413-81232013000500011,4,8052,4,0,0,1,0,0,0,0,1413-8123,scl,BR,1,2013


### Check

In [19]:
full_result[153234:154000].head(70)

,is_citable_x,year,origin_country,other_country,no_country,has_origin,has_other,has_no,has_both,all_no,issn,collection,origin,is_citable_y,years
pid,,,,,,,,,,,,,,,
S0102-64451993000100006,1,1993,0,1,0,0,1,0,0,0,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100007,1,1993,0,1,0,0,1,0,0,0,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100008,1,1993,1,0,0,1,0,0,0,0,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100009,1,1993,0,0,1,0,0,1,0,1,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100010,1,1993,0,1,0,0,1,0,0,0,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100011,1,1993,0,1,0,0,1,0,0,0,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100012,2,3986,2,0,0,1,0,0,0,0,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100013,1,1993,0,0,1,0,0,1,0,1,0102-6445,scl,BR,1,ate_2007
S0102-64451993000100014,1,1993,1,0,0,1,0,0,0,0,0102-6445,scl,BR,1,ate_2007


## Checking the result

There should be no more affiliations than what we had when we started... nor less...

In [20]:
full_result[["origin_country", "other_country", "no_country"]].values.sum() == cdfwof.shape[0]

True

In [21]:
full_result.shape

(377759, 15)

In [22]:
print(f"Notebook processing duration: {datetime.utcnow() - start}")

Notebook processing duration: 0:01:56.417162


## Pivot Table

###### filter by is_citable

In [23]:
filter_citables = full_result.loc[(full_result['is_citable_y'] == 1)]
filter_citables.shape

(343785, 15)

In [24]:
values_list = ["has_origin", "has_other", "has_no", "has_both", "all_no"]

td = filter_citables.pivot_table(
     index=["issn"],
     values=values_list,
     columns=["years"],
     aggfunc=np.count_nonzero,
     fill_value=0)

In [25]:
td.T

issn                 0001-3714  0001-3765  0002-0591  0004-2730  0004-2749  \
           years                                                             
all_no     2008              0          1          0          3          9   
           2009              0          4          0          5          8   
           2010              0          0          0          0          4   
           2011              0          0          0          3          2   
           2012              0          1         17          2          3   
           2013              0          0         20          5          5   
           2014              0          0          9          0          2   
           2015              0          0          0          0          0   
           2016              0          1          0          0          1   
           2017              0          2          0          0          0   
           2018              0          1          0          0          0   
           2019              0          0          0          0          0   
           ate_2007         16         36          0         89        135   
has_both   2008              0          8          0          8          8   
           2009              0          7          0          8          8   
           2010              0         13          0         14          3   
           2011              0         12          0          4          7   
           2012              0          5          0          7          7   
           2013              0         17          0          5          8   
           2014              0         11          0          9          5   
           2015              0         28          0          0          7   
           2016              0         21          0          0          7   
           2017              0         29          0          0         11   
           2018              0         25          0          0          9   
           2019              0          0          0          0          0   
           ate_2007          8         75          0         19         80   
has_no     2008              0          4          0         12         30   
           2009              0          6          0         16         38   
           2010              0          2          0          3         17   
           2011              0          3          0          3         11   
...                        ...        ...        ...        ...        ...   
           2017              0         14          0          0          1   
           2018              0          4          0          0          0   
           2019              0          0          0          0          0   
           ate_2007         37         71          0        263        418   
has_origin 2008              0         50          0        182        175   
           2009              0         70          0        137        150   
           2010              0         71          0        113        105   
           2011              0         95          0         86         90   
           2012              0         91          0         91         84   
           2013              0        146          0         75         80   
           2014              0        139          6         88         58   
           2015              0        167          0          0         54   
           2016              0        172          0          0         46   
           2017              0        265          0          0         53   
           2018              0        243          0          0         41   
           2019              0          0          0          0          0   
           ate_2007         75        384          0        749       1569   
has_other  2008              0         18          0         14         15   
           2009       

#### Renames the labels for CSV

In [26]:
# r is rename
r = {"has_origin":"pais_",
     "has_other":"estrang_",
     "has_no":"nao_ident_",
     "has_both":"pais_estrang_",
     "all_no":"nao_ident_todos_"
    }
newlabel = []
for k in td.keys():
    newlabel.append(r[k[0]]+k[1]) 

In [28]:
newlabel

['nao_ident_todos_2008',
 'nao_ident_todos_2009',
 'nao_ident_todos_2010',
 'nao_ident_todos_2011',
 'nao_ident_todos_2012',
 'nao_ident_todos_2013',
 'nao_ident_todos_2014',
 'nao_ident_todos_2015',
 'nao_ident_todos_2016',
 'nao_ident_todos_2017',
 'nao_ident_todos_2018',
 'nao_ident_todos_2019',
 'nao_ident_todos_ate_2007',
 'pais_estrang_2008',
 'pais_estrang_2009',
 'pais_estrang_2010',
 'pais_estrang_2011',
 'pais_estrang_2012',
 'pais_estrang_2013',
 'pais_estrang_2014',
 'pais_estrang_2015',
 'pais_estrang_2016',
 'pais_estrang_2017',
 'pais_estrang_2018',
 'pais_estrang_2019',
 'pais_estrang_ate_2007',
 'nao_ident_2008',
 'nao_ident_2009',
 'nao_ident_2010',
 'nao_ident_2011',
 'nao_ident_2012',
 'nao_ident_2013',
 'nao_ident_2014',
 'nao_ident_2015',
 'nao_ident_2016',
 'nao_ident_2017',
 'nao_ident_2018',
 'nao_ident_2019',
 'nao_ident_ate_2007',
 'pais_2008',
 'pais_2009',
 'pais_2010',
 'pais_2011',
 'pais_2012',
 'pais_2013',
 'pais_2014',
 'pais_2015',
 'pais_2016',
 'pa

In [29]:
td.columns = newlabel

In [30]:
td.head(9)

,nao_ident_todos_2008,nao_ident_todos_2009,nao_ident_todos_2010,nao_ident_todos_2011,nao_ident_todos_2012,nao_ident_todos_2013,nao_ident_todos_2014,nao_ident_todos_2015,nao_ident_todos_2016,nao_ident_todos_2017,...,estrang_2011,estrang_2012,estrang_2013,estrang_2014,estrang_2015,estrang_2016,estrang_2017,estrang_2018,estrang_2019,estrang_ate_2007
issn,,,,,,,,,,,,,,,,,,,,,
0001-3714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14
0001-3765,1,4,0,0,1,0,0,0,1,2,...,30,16,29,24,46,48,48,67,0,149
0002-0591,0,0,0,0,17,20,9,0,0,0,...,0,1,0,0,0,0,0,0,0,0
0004-2730,3,5,0,3,2,5,0,0,0,0,...,19,18,23,47,0,0,0,0,0,63
0004-2749,9,8,4,2,3,5,2,0,1,0,...,8,13,19,36,43,62,48,39,5,103
0004-2803,5,12,3,2,4,0,0,0,0,0,...,6,11,2,10,16,7,16,16,0,10
0004-282X,18,40,15,31,7,3,0,0,1,0,...,25,37,34,53,53,31,37,34,0,284
0006-8705,4,2,8,6,2,2,0,0,0,0,...,5,4,1,5,12,2,15,11,0,23
0011-5258,10,17,9,1,3,1,0,0,0,0,...,1,2,10,8,5,7,9,3,0,4


In [33]:
# td.to_csv("output/td_affi_network.csv")
td.to_csv("output/td_affi_bra_190123.csv")

In [34]:
print(f"Notebook processing duration: {datetime.utcnow() - start}")

Notebook processing duration: 0:03:46.696448
